In [11]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import keras
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
import nltk
import os
import warnings
from nltk.corpus import stopwords
import keras_tuner as kt
import tensorflow as tf

nltk.download('wordnet')
nltk.download('stopwords')
warnings.filterwarnings('ignore')

path = os.path.join(os.getcwd(), 'DATA/')
path_fn= path + 'rt-polarity.neg'
with open(path_fn, "r",encoding='utf-8', errors='ignore') as f:
    content_fn = f.read()  
texts_neg = content_fn.splitlines()
path_fp = path + 'rt-polarity.pos'
with open(path_fp, "r",encoding='utf-8', errors='ignore') as f:
    content_fp = f.read()  
texts_pos = content_fp.splitlines()
df = pd.DataFrame(texts_neg + texts_pos, columns=['Text'])
df['Target'] = 0
df['Target'][5332:] = 1

stop_words = set(stopwords.words('english'))
df['Text'] = df['Text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))

w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()

def lemmatize_text(text):
    
    st = ""
    for w in w_tokenizer.tokenize(text):
        st = st + lemmatizer.lemmatize(w) + " "
    return st
    
df['Text'] = df['Text'].apply(lemmatize_text)

X_train, X_test, y_train, y_test = train_test_split(df['Text'], df['Target'], stratify = df['Target'], random_state=42)

vocab_size = 1000
embedding_dim = 100
max_length = 50
padding_type='post'
trunc_type='post'

tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
word_index = tokenizer.word_index
train_sequences = tokenizer.texts_to_sequences(X_train)
train_padded = pad_sequences(train_sequences, padding='post', maxlen=max_length)
test_sequences = tokenizer.texts_to_sequences(X_test)
test_padded = pad_sequences(test_sequences, padding='post', maxlen=max_length)

def model_builder(hp):
    model = keras.Sequential([
        keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
        keras.layers.Bidirectional(keras.layers.LSTM(64)),
        keras.layers.Dense(24, activation='relu'),
        keras.layers.Dense(1, activation='sigmoid')
    ])
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    model.compile(loss='binary_crossentropy',
                  optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                  metrics=['accuracy'])
    return model

num_epochs = 50
history = model.fit(train_padded, y_train, 
                    epochs=num_epochs, verbose=1, 
                    validation_split=0.1)

tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=50,
                     factor=5,
                     directory='my_dir',
                     project_name='intro_to_kt')

stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5)

tuner.search(train_padded, y_train, epochs=50, validation_split=0.2, callbacks=[stop_early])


best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(best_hps)

[nltk_data] Downloading package wordnet to /home/sv/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/sv/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Epoch 1/50
225/225 [==============================] - 2s 8ms/step - loss: 0.0329 - accuracy: 0.9882 - val_loss: 2.7729 - val_accuracy: 0.6325
Epoch 2/50
225/225 [==============================] - 2s 8ms/step - loss: 0.0333 - accuracy: 0.9871 - val_loss: 2.5265 - val_accuracy: 0.6388
Epoch 3/50
225/225 [==============================] - 2s 7ms/step - loss: 0.0255 - accuracy: 0.9890 - val_loss: 2.8878 - val_accuracy: 0.6513
Epoch 4/50
225/225 [==============================] - 2s 7ms/step - loss: 0.0225 - accuracy: 0.9910 - val_loss: 2.9215 - val_accuracy: 0.6375
Epoch 5/50
225/225 [==============================] - 2s 7ms/step - loss: 0.0246 - accuracy: 0.9910 - val_loss: 2.9691 - val_accuracy: 0.6413
Epoch 6/50
225/225 [==============================] - 2s 7ms/step - loss: 0.0188 - accuracy: 0.9917 - val_loss: 3.0383 - val_accuracy: 0.6488
Epoch 7/50
225/225 [==============================] - 2s 7ms/step - loss: 0.0170 - accuracy: 0.9919 - val_loss: 3.2401 - val_accuracy: 0.6325
Epoch 

In [12]:
model = tuner.hypermodel.build(best_hps)
history = model.fit(train_padded, y_train, epochs=50, validation_split=0.2)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

Epoch 1/50
200/200 [==============================] - 12s 43ms/step - loss: 0.6417 - accuracy: 0.6152 - val_loss: 0.5684 - val_accuracy: 0.7088
Epoch 2/50
200/200 [==============================] - 3s 15ms/step - loss: 0.5232 - accuracy: 0.7400 - val_loss: 0.5540 - val_accuracy: 0.7262
Epoch 3/50
200/200 [==============================] - 2s 12ms/step - loss: 0.4800 - accuracy: 0.7703 - val_loss: 0.5649 - val_accuracy: 0.7275
Epoch 4/50
200/200 [==============================] - 2s 8ms/step - loss: 0.4482 - accuracy: 0.7897 - val_loss: 0.5793 - val_accuracy: 0.7225
Epoch 5/50
200/200 [==============================] - 2s 8ms/step - loss: 0.4152 - accuracy: 0.8118 - val_loss: 0.6067 - val_accuracy: 0.7050
Epoch 6/50
200/200 [==============================] - 2s 10ms/step - loss: 0.3803 - accuracy: 0.8282 - val_loss: 0.6859 - val_accuracy: 0.7050
Epoch 7/50
200/200 [==============================] - 2s 9ms/step - loss: 0.3498 - accuracy: 0.8416 - val_loss: 0.8018 - val_accuracy: 0.6875
E

In [13]:
hypermodel = tuner.hypermodel.build(best_hps)
hypermodel.fit(train_padded, y_train, epochs=best_epoch, validation_split=0.2)

Epoch 1/3
200/200 [==============================] - 11s 41ms/step - loss: 0.6419 - accuracy: 0.6132 - val_loss: 0.5646 - val_accuracy: 0.7169
Epoch 2/3
200/200 [==============================] - 3s 15ms/step - loss: 0.5234 - accuracy: 0.7417 - val_loss: 0.5505 - val_accuracy: 0.7212
Epoch 3/3
200/200 [==============================] - 2s 10ms/step - loss: 0.4870 - accuracy: 0.7689 - val_loss: 0.5709 - val_accuracy: 0.7250


In [15]:
eval_result = hypermodel.evaluate(test_padded, y_test)
print("[test loss, test accuracy]:", eval_result)

84/84 [==============================] - 0s 4ms/step - loss: 0.6038 - accuracy: 0.6740
[test loss, test accuracy]: [0.6038009524345398, 0.6740435361862183]
